In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from models import AlexNet, LeNet
from keras import backend as K
K.set_image_data_format('channels_last')

ModuleNotFoundError: No module named 'data_generator'

Image-treat-1: sem data augmentation, com normalização e equalização

Image-treat-2: com data augmentation, com normalização e equalização

Image-treat-3: com data augmentation, com normalização e com equalização

In [7]:
approach = 'image-treat-2' 
activation = 'relu'
net = 'alexnet'

if net == 'alexnet':
    model = AlexNet
elif net =='lenet':
    model = LeNet
csvlogger_name = 'callbacks/'+net +'/age/history-regression-' + approach + '-' + activation + '.csv'
checkpoint_filename = 'callbacks/'+net+'/age/class-weights-' + approach + '-' + activation + '.{epoch:02d}-{val_loss:.2f}.hdf5'
csvlogger_name, checkpoint_filename

('callbacks/lenet/age/history-regression-image-treat-2-lrelu.csv',
 'callbacks/lenet/age/class-weights-image-treat-2-lrelu.{epoch:02d}-{val_loss:.2f}.hdf5')

In [8]:
df = pd.read_csv('/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression_train_split_47950-70-10-20.csv')

In [9]:
cols = list(df.columns[1:])
in_format = list(df.columns)

In [10]:
train_dataset = BatchGenerator(box_output_format=cols)
validation_dataset = BatchGenerator(box_output_format=cols)

train_dataset. parse_csv(labels_filename='/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression_train_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/alexnet/dataset/imdb-hand-crop',
                        input_format=in_format)

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression_val_split_47950-70-10-20.csv', 
                             images_dir='/home/nicoli/github/alexnet/dataset/imdb-hand-crop',
                             input_format=in_format)

In [11]:
img_height, img_width, img_depth = (224,224,3)

epochs = 100

train_batch_size = 32
shuffle = True
ssd_train = False

validation_batch_size = 32


In [12]:
train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         random_rotation=20,
                                         translate=(0.2, 0.2),
                                         scale=(0.8, 1.2),
                                         flip=0.5,
                                         divide_by_stddev=255,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   divide_by_stddev=255,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())

Number of images in the dataset: 33565
Number of images in the dataset: 4795


In [13]:
steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size

In [14]:
model = model(n_classes=1, img_width=img_width, img_depth=img_depth, img_height=img_height, activation=activation)

/home/nicoli/github/alexnet/models.py:109: UserWarning: Considering a regression task with output function being lrelu
  warnings.warn(warning)


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 220, 220, 6)       456       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 220, 220, 6)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 6)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 106, 106, 16)      2416      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 106, 106, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 44944)             0         
__________

In [16]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=True)

In [17]:
csv_logger = CSVLogger(csvlogger_name, append=True, separator=',')

checkpoint = ModelCheckpoint(checkpoint_filename,
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=False,
                           period=1)

earlystopping = EarlyStopping(patience=5, mode='min')

#callbacks = [tensorboard, checkpoint]
callbacks=[checkpoint, csv_logger, earlystopping]

In [18]:
model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

In [ ]:
model.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps,
                             callbacks=callbacks)

Epoch 1/100
1049/1048 [==============================] - 3751s 4s/step - loss: 223.4216 - mean_absolute_error: 11.7186 - val_loss: 198.9249 - val_mean_absolute_error: 11.1937

Epoch 00001: saving model to callbacks/lenet/age/class-weights-image-treat-2-lrelu.01-198.92.hdf5
Epoch 2/100
1049/1048 [==============================] - 3712s 4s/step - loss: 207.4345 - mean_absolute_error: 11.3573 - val_loss: 193.5887 - val_mean_absolute_error: 10.9629

Epoch 00002: saving model to callbacks/lenet/age/class-weights-image-treat-2-lrelu.02-193.59.hdf5
Epoch 3/100
 601/1048 [================>.............] - ETA: 25:11 - loss: 204.8360 - mean_absolute_error: 11.2909